In [2]:
"""
A module for fitting the complex refractive index profile over a broad
bandwidth to a sum of Lorentzian polarizability terms using gradient-based
optimization via NLopt (nlopt.readthedocs.io). The fitting parameters are
then used to define a `Medium` object.
"""
from typing import Tuple

import matplotlib
import meep as mp
import nlopt
import numpy as np

matplotlib.use("agg")
import matplotlib.pyplot as plt


def lorentzfunc(p: np.ndarray, x: np.ndarray) -> np.ndarray:
    """
    Returns the complex ε profile given a set of Lorentzian parameters p
    (σ_0, ω_0, γ_0, σ_1, ω_1, γ_1, ...) for a set of frequencies x.
    """
    N = len(p) // 3
    y = np.zeros(len(x))
    for n in range(N):
        A_n = p[3 * n + 0]
        x_n = p[3 * n + 1]
        g_n = p[3 * n + 2]
        y = y + A_n / (np.square(x_n) - np.square(x) - 1j * x * g_n)
    return y


def lorentzerr(p: np.ndarray, x: np.ndarray, y: np.ndarray, grad: np.ndarray) -> float:
    """
    Returns the error (or residual or loss) as the L2 norm
    of the difference of ε(p,x) and y over a set of frequencies x as
    well as the gradient of this error with respect to each Lorentzian
    polarizability parameter in p and saving the result in grad.
    """
    N = len(p) // 3
    yp = lorentzfunc(p, x)
    val = np.sum(np.square(abs(y - yp)))
    for n in range(N):
        A_n = p[3 * n + 0]
        x_n = p[3 * n + 1]
        g_n = p[3 * n + 2]
        d = 1 / (np.square(x_n) - np.square(x) - 1j * x * g_n)
        if grad.size > 0:
            grad[3 * n + 0] = 2 * np.real(np.dot(np.conj(yp - y), d))
            grad[3 * n + 1] = (
                -4 * x_n * A_n * np.real(np.dot(np.conj(yp - y), np.square(d)))
            )
            grad[3 * n + 2] = (
                -2 * A_n * np.imag(np.dot(np.conj(yp - y), x * np.square(d)))
            )
    return val


def lorentzfit(
    p0: np.ndarray,
    x: np.ndarray,
    y: np.ndarray,
    alg=nlopt.LD_LBFGS,
    tol: float = 1e-25,
    maxeval: float = 10000,
) -> Tuple[np.ndarray, float]:
    """
    Returns the optimal Lorentzian polarizability parameters and error
    which minimize the error in ε(p0,x) relative to y for an initial
    set of Lorentzian polarizability parameters p0 over a set of
    frequencies x using the NLopt algorithm alg for a relative
    tolerance tol and a maximum number of iterations maxeval.
    """
    opt = nlopt.opt(alg, len(p0))
    opt.set_ftol_rel(tol)
    opt.set_maxeval(maxeval)
    opt.set_lower_bounds(np.zeros(len(p0)))
    opt.set_upper_bounds(float("inf") * np.ones(len(p0)))
    opt.set_min_objective(lambda p, grad: lorentzerr(p, x, y, grad))
    local_opt = nlopt.opt(nlopt.LD_LBFGS, len(p0))
    local_opt.set_ftol_rel(1e-10)
    local_opt.set_xtol_rel(1e-8)
    opt.set_local_optimizer(local_opt)
    popt = opt.optimize(p0)
    minf = opt.last_optimum_value()
    return popt, minf


if __name__ == "__main__":
    # Import the complex refractive index profile from a CSV file.
    # The file format is three comma-separated columns:
    #     wavelength (nm), real(n), imag(n).
    mydata = np.genfromtxt("mymaterial.csv", delimiter=",")
    n = mydata[:, 1] + 1j * mydata[:, 2]

    # Fitting parameter: the instantaneous (infinite frequency) dielectric.
    # Should be > 1.0 for stability and chosen such that
    # np.amin(np.real(eps)) is ~1.0. eps is defined below.
    eps_inf = 1.1

    eps = np.square(n) - eps_inf

    # Fit only the data in the wavelength range of [wl_min, wl_max].
    wl = mydata[:, 0]
    wl_min = 399  # minimum wavelength (units of nm)
    wl_max = 701  # maximum wavelength (units of nm)
    start_idx = np.where(wl > wl_min)
    idx_start = start_idx[0][0]
    end_idx = np.where(wl < wl_max)
    idx_end = end_idx[0][-1] + 1

    # The fitting function is ε(f) where f is the frequency, rather than ε(λ).
    # Note: an equally spaced grid of wavelengths results in the larger
    #       wavelengths having a finer frequency grid than smaller ones.
    #       This feature may impact the accuracy of the fit.
    freqs = 1000 / wl  # units of 1/μm
    freqs_reduced = freqs[idx_start:idx_end]
    wl_reduced = wl[idx_start:idx_end]
    eps_reduced = eps[idx_start:idx_end]

    # Fitting parameter: number of Lorentzian terms to use in the fit
    num_lorentzians = 2

    # Number of times to repeat local optimization with random initial values.
    num_repeat = 30

    ps = np.zeros((num_repeat, 3 * num_lorentzians))
    mins = np.zeros(num_repeat)
    for m in range(num_repeat):
        # Initial values for the Lorentzian polarizability terms. Each term
        # consists of three parameters (σ, ω, γ) and is chosen randomly.
        # Note: for the case of no absorption, γ should be set to zero.
        p_rand = [10 ** (np.random.random()) for _ in range(3 * num_lorentzians)]
        ps[m, :], mins[m] = lorentzfit(
            p_rand, freqs_reduced, eps_reduced, nlopt.LD_MMA, 1e-25, 50000
        )
        ps_str = "( " + ", ".join(f"{prm:.4f}" for prm in ps[m, :]) + " )"
        print(f"iteration:, {m:3d}, ps_str, {mins[m]:.6f}")

    # Find the best performing set of parameters.
    idx_opt = np.where(np.min(mins) == mins)[0][0]
    popt_str = "( " + ", ".join(f"{prm:.4f}" for prm in ps[idx_opt]) + " )"
    print(f"optimal:, {popt_str}, {mins[idx_opt]:.6f}")

    # Define a `Medium` class object using the optimal fitting parameters.
    E_susceptibilities = []

    for n in range(num_lorentzians):
        mymaterial_freq = ps[idx_opt][3 * n + 1]
        mymaterial_gamma = ps[idx_opt][3 * n + 2]

        if mymaterial_freq == 0:
            mymaterial_sigma = ps[idx_opt][3 * n + 0]
            E_susceptibilities.append(
                mp.DrudeSusceptibility(
                    frequency=1.0, gamma=mymaterial_gamma, sigma=mymaterial_sigma
                )
            )
        else:
            mymaterial_sigma = ps[idx_opt][3 * n + 0] / mymaterial_freq**2
            E_susceptibilities.append(
                mp.LorentzianSusceptibility(
                    frequency=mymaterial_freq,
                    gamma=mymaterial_gamma,
                    sigma=mymaterial_sigma,
                )
            )

    mymaterial = mp.Medium(epsilon=eps_inf, E_susceptibilities=E_susceptibilities)

    # Plot the fit and the actual data for comparison.
    mymaterial_eps = [mymaterial.epsilon(f)[0][0] for f in freqs_reduced]

    fig, ax = plt.subplots(ncols=2)

    ax[0].plot(wl_reduced, np.real(eps_reduced) + eps_inf, "bo-", label="actual")
    ax[0].plot(wl_reduced, np.real(mymaterial_eps), "ro-", label="fit")
    ax[0].set_xlabel("wavelength (nm)")
    ax[0].set_ylabel(r"real($\epsilon$)")
    ax[0].legend()

    ax[1].plot(wl_reduced, np.imag(eps_reduced), "bo-", label="actual")
    ax[1].plot(wl_reduced, np.imag(mymaterial_eps), "ro-", label="fit")
    ax[1].set_xlabel("wavelength (nm)")
    ax[1].set_ylabel(r"imag($\epsilon$)")
    ax[1].legend()

    fig.suptitle(
        f"Comparison of Actual Material Data and Fit\n"
        f"using Drude-Lorentzian Susceptibility"
    )

    fig.subplots_adjust(wspace=0.3)
    fig.savefig("eps_fit_sample.png", dpi=150, bbox_inches="tight")

FileNotFoundError: mymaterial.csv not found.

In [ ]:
"""
A module for fitting the complex refractive index profile over a broad
bandwidth to a sum of Lorentzian polarizability terms using gradient-based
optimization via NLopt (nlopt.readthedocs.io). The fitting parameters are
then used to define a `Medium` object.
"""

from typing import Tuple

import matplotlib
import meep as mp
import nlopt
import numpy as np

# 백엔드 설정(Agg는 GUI 없이도 이미지를 생성 가능)
matplotlib.use("agg")
import matplotlib.pyplot as plt


def lorentzfunc(p: np.ndarray, x: np.ndarray) -> np.ndarray:
    """
    Returns the complex ε profile given a set of Lorentzian parameters p
    (σ_0, ω_0, γ_0, σ_1, ω_1, γ_1, ...) for a set of frequencies x.

    각 로렌츠 항에 대해:
        - A_n(= σ_n)은 진폭(또는 강도),
        - x_n(= ω_n)은 공진 주파수,
        - g_n(= γ_n)은 감쇠(감쇠율)를 나타냄.
    p: 로렌츠 모델의 파라미터들을 3개 단위(σ, ω, γ)로 묶어 1차원 배열에 나열
    x: 주파수(또는 각주파수) 목록
    반환값: 주어진 x에서의 복소 유전율(ε) 값
    """
    # 로렌츠 항의 개수 N
    N = len(p) // 3
    # 결과를 저장할 배열 (초기값 0으로)
    y = np.zeros(len(x), dtype=complex)  # 복소수이므로 dtype=complex로 보강할 수 있음

    # 각 로렌츠 항에 대해 누적합
    for n in range(N):
        A_n = p[3 * n + 0]  # σ_n
        x_n = p[3 * n + 1]  # ω_n
        g_n = p[3 * n + 2]  # γ_n

        # 로렌츠 항: A_n / (ω_n^2 - x^2 - i*x*γ_n)
        # (여기서 x는 실제로 'f(주파수)'에 해당)
        y += A_n / (x_n**2 - x**2 - 1j * x * g_n)

    return y


def lorentzerr(p: np.ndarray, x: np.ndarray, y: np.ndarray, grad: np.ndarray) -> float:
    """
    Returns the error (or residual or loss) as the L2 norm
    of the difference of ε(p,x) and y over a set of frequencies x as
    well as the gradient of this error with respect to each Lorentzian
    polarizability parameter in p and saving the result in grad.

    p: 로렌츠 모델의 현재 파라미터들
    x: 주파수 배열
    y: 목표 유전율(실험 혹은 실제 값) 배열
    grad: loss 함수에 대한 p의 기울기(gradient)를 저장할 배열 (NLopt가 사용)

    반환값:
      - 현재 파라미터 p에 대한 L2 오차 (실수 스칼라)
    """

    # 로렌츠 항의 개수
    N = len(p) // 3

    # 현재 파라미터에 기반한 유전율 예측값
    yp = lorentzfunc(p, x)

    # 오차 계산(L2 norm): sum(|y - yp|^2)
    # 복소수 절댓값 => np.square(abs(...)) 가능
    val = np.sum(np.square(np.abs(y - yp)))

    # grad가 실제로 필요할 때(optimizer가 gradient 계산을 요청할 때)
    if grad.size > 0:
        # 각 로렌츠 항(σ, ω, γ)에 대한 편미분을 구해 grad에 대입
        for n in range(N):
            A_n = p[3 * n + 0]  # σ_n
            x_n = p[3 * n + 1]  # ω_n
            g_n = p[3 * n + 2]  # γ_n

            # d = 1 / (ω_n^2 - x^2 - i*x*γ_n)
            d = 1.0 / (x_n**2 - x**2 - 1j * x * g_n)

            # grad[3*n+0]: σ_n에 대한 편미분
            #   => d/dσ_n (2 * Re((yp - y)^* · (∂yp/∂σ_n)))
            #   여기서 (yp - y)^* 은 켤레복소수
            grad[3 * n + 0] = 2 * np.real(np.dot(np.conj(yp - y), d))

            # grad[3*n+1]: ω_n에 대한 편미분
            #   ∂yp/∂ω_n = - A_n * 2ω_n / ( (ω_n^2 - x^2 - i*x*γ_n)^2 ) (부호 주의)
            #   실제 계산은 d^2(d^2 = d*d) 등을 통해 이루어짐
            grad[3 * n + 1] = -4 * x_n * A_n * np.real(
                np.dot(np.conj(yp - y), d**2)
            )

            # grad[3*n+2]: γ_n에 대한 편미분
            #   ∂yp/∂γ_n = - i * A_n * x / ( (ω_n^2 - x^2 - i*x*γ_n)^2 )
            #   여기서는 허수부 부분만 실질적으로 오차 기여 가능
            grad[3 * n + 2] = -2 * A_n * np.imag(
                np.dot(np.conj(yp - y), x * d**2)
            )

    return val


def lorentzfit(
    p0: np.ndarray,
    x: np.ndarray,
    y: np.ndarray,
    alg=nlopt.LD_LBFGS,
    tol: float = 1e-25,
    maxeval: float = 10000,
) -> Tuple[np.ndarray, float]:
    """
    Returns the optimal Lorentzian polarizability parameters and error
    which minimize the error in ε(p0,x) relative to y for an initial
    set of Lorentzian polarizability parameters p0 over a set of
    frequencies x using the NLopt algorithm alg for a relative
    tolerance tol and a maximum number of iterations maxeval.

    p0: 초기 추정 파라미터 배열
    x:  주파수 배열
    y:  목표 유전율 배열
    alg: NLopt 알고리즘 종류 (기본값: LBFGS)
    tol: 목표 상대 오차 (컨버전스 기준)
    maxeval: 최대 반복 횟수

    반환값:
      popt: 최적화된 파라미터
      minf: 해당 파라미터에서의 목표함수(오차) 값
    """
    # nlopt.optimizer 생성
    opt = nlopt.opt(alg, len(p0))

    # 설정들
    opt.set_ftol_rel(tol)         # 목표 함수의 상대 변화 허용 오차
    opt.set_maxeval(maxeval)      # 최대 반복 횟수

    # 파라미터의 하한, 상한 설정(0 ~ +∞)
    opt.set_lower_bounds(np.zeros(len(p0)))
    opt.set_upper_bounds(float("inf") * np.ones(len(p0)))

    # 최소화할 목적함수 설정
    opt.set_min_objective(lambda p, grad: lorentzerr(p, x, y, grad))

    # 지역(local) 최적화 도구 설정 (멀티레벨 방식으로 사용 가능)
    local_opt = nlopt.opt(nlopt.LD_LBFGS, len(p0))
    local_opt.set_ftol_rel(1e-10)
    local_opt.set_xtol_rel(1e-8)

    opt.set_local_optimizer(local_opt)

    # 최적화 실행
    popt = opt.optimize(p0)            # 파라미터 p0에서 시작
    minf = opt.last_optimum_value()    # 최적화 후의 목적함수값

    return popt, minf


if __name__ == "__main__":
    # mymaterial.csv 파일에서 다음 3개 열(컬럼)을 불러옴:
    #   1) wavelength (nm), 2) real(n), 3) imag(n)
    #   => n = real(n) + i*imag(n)
    mydata = np.genfromtxt("mymaterial.csv", delimiter=",")
    n = mydata[:, 1] + 1j * mydata[:, 2]

    # eps_inf: 무한 고주파 영역에서의 유전율 (일종의 상수 offset 개념).
    #          np.amin(np.real(eps))을 약 1.0 정도로 맞추도록 설정 권장
    eps_inf = 1.1

    # eps = n^2 - eps_inf
    #   => 실제 유전율에서 eps_inf를 빼 준 것을 fitting 대상 값으로 사용
    eps = np.square(n) - eps_inf

    # 피팅 범위를 설정([wl_min, wl_max] 구간만 활용)
    wl = mydata[:, 0]  # 파장 (nm 단위)
    wl_min = 399
    wl_max = 701

    # wl_min 보다 큰 구간에서의 시작 인덱스
    start_idx = np.where(wl > wl_min)
    idx_start = start_idx[0][0]

    # wl_max 보다 작은 구간에서의 끝 인덱스
    end_idx = np.where(wl < wl_max)
    idx_end = end_idx[0][-1] + 1

    # meep에서는 주로 주파수(f)를 쓰기 때문에,
    # f = 1000 / λ (단위 보정: nm -> μm 를 고려한 스케일로 추정)
    # wl이 nm 단위이고, 보통 meep의 길이 단위는 μm
    freqs = 1000 / wl   # (1/μm)

    # 관심 구간에 해당하는 부분 슬라이싱
    freqs_reduced = freqs[idx_start:idx_end]
    wl_reduced = wl[idx_start:idx_end]
    eps_reduced = eps[idx_start:idx_end]

    # 로렌츠 항의 개수 (예: 2개)
    num_lorentzians = 2

    # 랜덤한 초기값으로 local optimization을 반복할 횟수
    num_repeat = 30

    # 결과 저장용 배열
    ps = np.zeros((num_repeat, 3 * num_lorentzians))
    mins = np.zeros(num_repeat)

    # 여러 번 반복하여 랜덤 초기값 중 가장 좋은 해를 찾음
    for m in range(num_repeat):
        # 초기값(σ, ω, γ)을 랜덤하게 생성 (10^(0~1 사이의 랜덤))
        p_rand = [10 ** (np.random.random()) for _ in range(3 * num_lorentzians)]

        # lorentzfit 실행 (이번 예시에서는 NLopt의 LD_MMA 알고리즘 사용)
        ps[m, :], mins[m] = lorentzfit(
            p_rand, freqs_reduced, eps_reduced, nlopt.LD_MMA, 1e-25, 50000
        )

        ps_str = "( " + ", ".join(f"{prm:.4f}" for prm in ps[m, :]) + " )"
        print(f"iteration:, {m:3d}, ps_str, {mins[m]:.6f}")

    # 가장 에러가 작은 파라미터 찾기
    idx_opt = np.where(np.min(mins) == mins)[0][0]
    popt_str = "( " + ", ".join(f"{prm:.4f}" for prm in ps[idx_opt]) + " )"
    print(f"optimal:, {popt_str}, {mins[idx_opt]:.6f}")

    # Meep에서 사용할 Medium 객체 정의
    E_susceptibilities = []

    for n in range(num_lorentzians):
        mymaterial_freq = ps[idx_opt][3 * n + 1]  # ω_n
        mymaterial_gamma = ps[idx_opt][3 * n + 2] # γ_n

        # ω_n이 0이면 DrudeSusceptibility 사용(DC 항?)
        if mymaterial_freq == 0:
            mymaterial_sigma = ps[idx_opt][3 * n + 0]
            E_susceptibilities.append(
                mp.DrudeSusceptibility(
                    frequency=1.0,
                    gamma=mymaterial_gamma,
                    sigma=mymaterial_sigma
                )
            )
        else:
            # 로렌츠 항의 sigma = A_n / (ω_n^2)
            mymaterial_sigma = ps[idx_opt][3 * n + 0] / mymaterial_freq**2
            E_susceptibilities.append(
                mp.LorentzianSusceptibility(
                    frequency=mymaterial_freq,
                    gamma=mymaterial_gamma,
                    sigma=mymaterial_sigma,
                )
            )

    # mp.Medium 객체 생성
    mymaterial = mp.Medium(epsilon=eps_inf, E_susceptibilities=E_susceptibilities)

    # 피팅 결과 vs 실제 데이터를 비교 플롯
    mymaterial_eps = [mymaterial.epsilon(f)[0][0] for f in freqs_reduced]

    fig, ax = plt.subplots(ncols=2)

    # 실수부: 실제값 vs 피팅값
    ax[0].plot(wl_reduced, np.real(eps_reduced) + eps_inf, "bo-", label="actual")
    ax[0].plot(wl_reduced, np.real(mymaterial_eps), "ro-", label="fit")
    ax[0].set_xlabel("wavelength (nm)")
    ax[0].set_ylabel(r"real($\epsilon$)")
    ax[0].legend()

    # 허수부: 실제값 vs 피팅값
    ax[1].plot(wl_reduced, np.imag(eps_reduced), "bo-", label="actual")
    ax[1].plot(wl_reduced, np.imag(mymaterial_eps), "ro-", label="fit")
    ax[1].set_xlabel("wavelength (nm)")
    ax[1].set_ylabel(r"imag($\epsilon$)")
    ax[1].legend()

    fig.suptitle(
        f"Comparison of Actual Material Data and Fit\n"
        f"using Drude-Lorentzian Susceptibility"
    )

    fig.subplots_adjust(wspace=0.3)
    fig.savefig("eps_fit_sample.png", dpi=150, bbox_inches="tight")


In [3]:
import asammdf

def convert_large_mdf_in_chunks(mdf_file_path: str, csv_file_path: str, chunk_size: int = 100000):
    mdf = asammdf.MDF(mdf_file_path)
    
    # chunk_size: 한 번에 몇 개의 데이터 포인트씩 읽을지 결정
    # 'time_as_date'나 'use_display_names' 등의 옵션도 사용 가능
    for chunk_df in mdf.to_dataframe(chunksize=chunk_size):
        chunk_df.to_csv(csv_file_path, mode='a', header=not bool(chunk_df.index.name), index=False)


mdf_file_path = r'/home/min/EIDL/Tool/Meep/LGD/Al_Palik.mdf'
csv_file_path = r'/home/min/EIDL/Tool/Meep/LGD/Material CSV'
convert_large_mdf_in_chunks(mdf_file_path, csv_file_path, chunk_size=100000)


MdfException: "/home/min/EIDL/Tool/Meep/LGD/Al_Palik.mdf" is not a valid ASAM MDF file: magic header is b'Lumerica'

In [5]:
import asammdf

mdf_file_path = r'/home/min/EIDL/Tool/Meep/LGD/Al_Palik.mdf'
csv_file_path = r'/home/min/EIDL/Tool/Meep/LGD/Material CSV'

# MDF 파일 열기
mdf = asammdf.MDF(mdf_file_path)

# MDF 내용을 판다스 DataFrame으로 변환
df = mdf.to_dataframe()

# CSV 파일로 저장
df.to_csv(csv_file_path, index=False)

print(f"변환 완료: {csv_file_path}")

MdfException: "/home/min/EIDL/Tool/Meep/LGD/Al_Palik.mdf" is not a valid ASAM MDF file: magic header is b'Lumerica'